In [108]:
from gui import GUI
import numpy as np

In [109]:
image_f = 'fire_C11659/S2B_MSIL2A_20251009T192229_N0511_R099_T09UYU_20251009T231731_cloudfree.bin_MRAP_C11659.bin'
polygon_f = 'polygon_C11659/polygon_0000.bin'

In [110]:
g = GUI(
    polygon_filename = polygon_f,
    image_filename = image_f
)

running: [1, 3]: pid = 44981.
running: [1, 4]: pid = 44982.
running: [1, 2]: pid = 44980.
running: [1, 2, 3]: pid = 44988.
running: [2, 3]: pid = 44984.
running: [3, 4]: pid = 44983.
running: [1, 2, 3, 4]: pid = 44990.
running: [1, 2, 4]: pid = 44989.
running: [2, 3, 4]: pid = 44986.
running: [2, 4]: pid = 44985.
running: [1, 3, 4]: pid = 44987.


KeyboardInterrupt: 

In [ ]:
g.band_list = [1,2,3]

In [ ]:
current_embeds = g.get_band_embed()

In [ ]:
g.in_sample_size

500

In [ ]:
inside = current_embeds[:g.in_sample_size]
outside= current_embeds[g.in_sample_size:]

**Which cluster is closer?**

We want to find Eucledian distance of each point to each cluster mean.

E.g Point 1 of cluster 1 to cluster 1 vs cluster 2, which is closer?

In [ ]:
inside_mean = np.mean(inside, axis=0)
outside_mean= np.mean(outside,axis=0)

**Eucledian distance**

In [ ]:
inside_mean

array([-0.72418998,  1.52829674])

In [ ]:
#Inside data: to the means

d_in_in = (inside[:, 0] - inside_mean[0]) ** 2 + (inside[:, 1] - inside_mean[1]) ** 2
d_in_out = (inside[:, 0] - outside_mean[0]) ** 2 + (inside[:, 1] - outside_mean[1]) ** 2

In [ ]:
#Outside data: to the means

d_out_in  = (outside[:, 0] - inside_mean[0]) ** 2 + (outside[:, 1] - inside_mean[1]) ** 2
d_out_out = (outside[:, 0] - outside_mean[0]) ** 2 + (outside[:, 1] - outside_mean[1]) ** 2

**So, would point of cluster Inside be closer to its cluster mean than Outside's?**

How many percent of inside samples are really closer to its cluster?

In [ ]:
np.mean(d_in_in > d_in_out)

np.float64(0.154)

In [ ]:
np.mean(d_out_out > d_out_in)

np.float64(0.19738562091503267)

**Differencing**

In [111]:
from change_detection import change_detection
from raster import Raster

In [112]:
pre = Raster('fire_C11659/S2C_MSIL2A_20250609T192931_N0511_R142_T09UYU_20250610T002612_cloudfree.bin_MRAP_C11659.bin')
pre_dat = pre.read_bands()

In [114]:
post = Raster('fire_C11659/S2B_MSIL2A_20251009T192229_N0511_R099_T09UYU_20251009T231731_cloudfree.bin_MRAP_C11659.bin')
post_dat = post.read_bands()

In [115]:
change = change_detection(
    pre_X=pre_dat,
    post_X=post_dat
)

In [116]:
from misc.files import writeENVI

In [118]:
writeENVI(
    'S2F_MSIL2A_20250000000000000.bin',
    'fire_C11659/S2B_MSIL2A_20250830T191909_N0511_R099_T09UYU_20250830T225737_cloudfree.bin_MRAP_C11659.bin',
    change
)